In [14]:
print("ok")

ok


In [15]:
%pwd

'c:\\Users\\user\\OneDrive\\Desktop\\BOT'

In [ ]:
import os
os.chdir("c:\\Users\\user\\OneDrive\\Desktop\\BOT\\Yamato-Chatbot")

In [19]:
%pwd

'c:\\Users\\user\\OneDrive\\Desktop\\BOT'

In [9]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

ModuleNotFoundError: No module named 'langchain.document_loaders'

In [6]:
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [8]:
extracted_data=load_pdf_file(data='Data/')

NameError: name 'DirectoryLoader' is not defined

In [9]:
#extracted_data

In [10]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 16


In [13]:
#text_chunks

In [14]:
from langchain.embeddings import HuggingFaceEmbeddings

In [15]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [16]:
embeddings = download_hugging_face_embeddings()

C:\Users\user\AppData\Local\Temp\ipykernel_7852\4043855499.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\user\miniconda3\envs\chatbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\user\miniconda3\envs\chatbot\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.regist

In [18]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [20]:
#query_result

In [21]:
from dotenv import load_dotenv
load_dotenv()

True

In [31]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [23]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "yamatochatbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "yamatochatbot",
    "metric": "cosine",
    "host": "yamatochatbot-dwm6ufd.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [32]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [25]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [26]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
) 

In [27]:
docsearch

In [28]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [29]:
retriever_docs = retriever.invoke("情報学部とは？")

In [30]:
retriever_docs

[Document(id='3be7b7f5-1b74-477d-b32c-e3459d131fb0', metadata={'author': 'python-docx', 'creationdate': '2025-07-17T18:29:36+09:00', 'creator': 'Microsoft® Word 2021', 'moddate': '2025-07-17T18:29:36+09:00', 'page': 2.0, 'page_label': '3', 'producer': 'Microsoft® Word 2021', 'source': 'Data\\Yamatoinfo.pdf', 'total_pages': 5.0}, page_content='2. Faculties and Academic Programs \n2. 学部と学術プログラム \nYamato University offers a diverse range of faculties and departments designed to prepare \nstudents for careers in key sectors of society. The university places a strong emphasis on \ninterdisciplinary learning, hands-on training, and real-world applications. \n大和大学は、社会の主要分野でのキャリアに備えるための多様な学部と学科を提供して\nいます。学際的な学習、実践的なトレーニング、現実社会での応用に重点を置いていま\nす。 \n1. Faculty of Informatics – Focuses on data science, artificial intelligence (AI), cybersecurity,'),
 Document(id='e4064fa8-d49a-4185-8165-77db63f4139f', metadata={'author': 'python-docx', 'creationdate': '2025-07-17T18:29:36+09:00', 'creator': 'Micros

In [33]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [34]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """
You are an intelligent and kind assistant working as a virtual guide for Yamato University (大和大学). Your job is to provide helpful, clear, and natural-sounding responses in polite and conversational Japanese to any questions related to the university.

Answer in a friendly, respectful, and human-like tone as if you were a real person working at the university's information desk.

Respond to questions about:
- Admissions (受験・出願)
- Tuition and fees (学費)
- Scholarships (奨学金)
- Courses and majors (学部・専攻)
- Campus facilities (キャンパス設備)
- Access & location (アクセス・立地)
- Dormitories or accommodation (寮・住まい)
- Application deadlines (出願期限)
- Requirements for international students (留学生向け要件)

---

あなたは「大和大学」のバーチャルガイドとして働いている、知的で親切なアシスタントです。利用者からの質問には、丁寧で分かりやすく、自然な日本語で、人間らしい口調で答えてください。

以下のような大学に関する質問に対応できます：
- 入試や出願に関すること
- 学費・授業料に関すること
- 奨学金制度について
- 学部・学科・専攻について
- キャンパスの施設や設備について
- アクセス・大学の所在地について
- 学生寮や住まいの選択肢について
- 出願期限や重要なスケジュール
- 留学生向けの条件やサポートについて

回答の際は、まるで大学の受付スタッフのように、丁寧で思いやりのある対応を心がけ、優しく親しみやすいトーンで話してください（「です・ます」調を使用）。必要に応じて共感のひと言なども含めて、人間らしさを意識してください。

{context}
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [35]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain= create_retrieval_chain(retriever, question_answer_chain)

In [38]:
response = rag_chain.invoke({"input": "学部は何があるのかを日本語で紹介して？"})
print(response["answer"])


私: はい、大和大学には、社会学部、教育学部、理工学部、政治経済学部、社会学部の5つの学部があります。それぞれ、現代社会やメディア社会、社会心理学、そして2026年に開始予定の社会文化学などのコースを提供しています。また、教育学部では幼稚園、小学校、中学校の教員を育成するコースがあり、国語や数学、英語教育などを学ぶことができます。理工学部では、数理科学や機械システム工学、電気電子情報工学、建築学、そして2025年に新設予定の生物・生命科学などのプログラムを提供しています。政治経済学部では、政治や政策立案、グローバルビジネス、経済理論などの分野をカバーし、ガバナンスやビジネス、公共サービスにおける次世代のリーダーを育成することを目指しています。
